取得向量資料庫的zip檔(不知道為什麼wget下載不下來，改用gdown)

In [12]:
!pip install gdown

In [13]:
import gdown

url = "https://drive.google.com/uc?export=download&id=1d6IEhQg4wZ9qdtOzo1OPNHfKPGbYfjpV"
output = "faiss_db.zip"
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1d6IEhQg4wZ9qdtOzo1OPNHfKPGbYfjpV
From (redirected): https://drive.google.com/uc?export=download&id=1d6IEhQg4wZ9qdtOzo1OPNHfKPGbYfjpV&confirm=t&uuid=e252f1af-580d-4799-998b-bb7683d6f70e
To: /content/faiss_db.zip
100%|██████████| 35.1M/35.1M [00:01<00:00, 27.8MB/s]


'faiss_db.zip'

In [11]:
# url = "https://drive.google.com/uc?export=download&id=1d6IEhQg4wZ9qdtOzo1OPNHfKPGbYfjpV"
# !wget -O faiss_db.zip "$url"

--2025-04-10 18:33:25--  https://drive.google.com/uc?export=download&id=1d6IEhQg4wZ9qdtOzo1OPNHfKPGbYfjpV
Resolving drive.google.com (drive.google.com)... 108.177.12.139, 108.177.12.138, 108.177.12.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.12.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1d6IEhQg4wZ9qdtOzo1OPNHfKPGbYfjpV&export=download [following]
--2025-04-10 18:33:25--  https://drive.usercontent.google.com/download?id=1d6IEhQg4wZ9qdtOzo1OPNHfKPGbYfjpV&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.134.132, 2607:f8b0:400c:c1a::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.134.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2423 (2.4K) [text/html]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]   2.37K  --.-KB/s    in 0s      

202

解壓縮

In [14]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
   creating: faiss_db/
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


### 1. 安裝並引入必要套件

In [6]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7

In [15]:
from langchain_community.vectorstores import FAISS        # 使用Langchain 的字串分割與Faiss 去做相似度搜尋
from langchain_community.embeddings import HuggingFaceEmbeddings # 用Hugging Face模型去生成文字向量
from langchain.chat_models import ChatOpenAI           # 在langchain中使用caht models
from langchain.chains import ConversationalRetrievalChain     # 建構更複雜的chains套件

In [16]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [17]:
# 繼承HuggingFaceEmbeddings，使得能進一步修改
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]  # 要求文件加上"passage:"前綴
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")  # 要求文件加上"query: "前綴

### 3. 載入 `faiss_db`

In [18]:
# 創建E5-large模型
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-large")
# 選擇faiss作為db對向量進行存儲
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
# 向量資料庫檢索器: 選定向量資料庫db來進行相似度搜尋
retriever = db.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

### 4. 設定好我們要的 LLM

In [19]:
import os
from google.colab import userdata

如之前, 我們會用 OpenAI API。這裡使用 Groq 服務, 可改成你要的服務。

In [20]:
api_key = userdata.get('Groq')

In [21]:
os.environ["OPENAI_API_KEY"] = api_key

這裡的模型和 `base_url` 是用 Groq, 如果用其他服務請自行修改。

In [22]:
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

In [23]:
client = OpenAI(
    base_url=base_url # 使用 OpenAI 本身不需要這段
)

### 5. `prompt` 設計

In [27]:
# 定義常見地區名單
taiwan_regions = ["台南", "台北", "台中", "高雄", "桃園", "新竹", "苗栗", "彰化", "雲林", "嘉義", "屏東", "宜蘭", "花蓮", "台東", "基隆", "新北", "南投", "中壢", "澎湖", "金門", "連江", "香港", "日本"]

system_prompt = """
你是台灣頂級美食鑑賞家，專精全台隱藏美味與經典名店，只根據資料庫的真實食記回應。以老饕的熱情與專業眼光，用純正台灣慣用中文（絕對不准用英文！），給出接地氣又垂涎三尺的建議。回答時必須嚴格符合使用者問的地區（例如台南就給台南，台北就給台北），若資料沒提到該地區的店，直接說沒找到，别亂推其他地方或瞎編！
"""

prompt_template = """
嘿，老饕的味蕾雷達啟動啦！以下是從美食寶庫挖出的情報：
{retrieved_chunks}

你的美食挑戰是：{question}

老饕我會根據這些真材實料的資料，給你最道地的推薦，地區要完全對上你的要求！全程用台灣中文，絕不夾雜英文。如果情報不夠，或沒找到你問的地區的店，就老實說「哎呀，這寶庫裡暫時沒這地區的貨色，建議問問偉大的Google老師，或學務處課外活動組的美食嚮導！」。
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [28]:
chat_history = []  # 初始化一個空列表用來儲存歷史聊天紀錄

# 定義使用rag回覆的函數
def chat_with_rag(user_input):
    global chat_history  # 宣告chat_history是全域變數，函數內可以修改它。
    # 取回相關資料
    retriever = db.as_retriever(search_kwargs={"k": 8})  # 8個片段抓取更多餐廳資訊
    docs = retriever.get_relevant_documents(user_input)   # 用retriever（FAISS的檢索器）根據使用者輸入找相關文件。

    # 提取問題中的地區
    region = None
    for r in taiwan_regions:
        if r in user_input:
            region = r
            break

    # 如果有指定地區，過濾資料
    if region:
        filtered_docs = [doc for doc in docs if region in doc.page_content]
        if not filtered_docs:
            return f"哎呀，這寶庫裡暫時沒{region}的貨色，建議問問偉大的Google老師！"
        retrieved_chunks = "\n\n".join([doc.page_content for doc in filtered_docs])
    else:
        # 沒指定地區就給全部
        retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": final_prompt},
        ]
    )
    answer = response.choices[0].message.content
    chat_history.append((user_input, answer))
    return answer

### 7. 用 Gradio 打造 Web App

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🍤 老饕的台灣美食寶庫 🔥\n探索無數真實食記內容，保證滿足你的味蕾挑戰！~")
    with gr.Row():
        # 使用者輸入框大小
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=850, show_label=False) # 歷史訊息呈現框的大小
            msg = gr.Textbox(placeholder="丟個美食挑戰給老饕吧（按shift+enter上傳對話）...", lines=2) # 使用者輸入框的提示文字
        # 清空紀錄的按鈕設定
        with gr.Column(scale=1):
            gr.Markdown("### 饕客小助手")
            clear_btn = gr.Button("清空味蕾記憶")
    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local
    def clear_history():
        global chat_history
        chat_history = []
        return []
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(clear_history, None, chatbot)
demo.launch(debug=True)

<ipython-input-32-6185c02a4f54>:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=850, show_label=False)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f4d167429610707824.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
